# RadCAD Model

In [1]:
import pandas as pd
import numpy as np
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend
#importing radcad model from models folder
from models.infinite_runner import model, simulation, experiment

In [2]:
# printing just to check if the model is working
df = pd.DataFrame(experiment.run())
df.head()

,distance,coins,difficulty_factor,player_crashes,simulation,subset,run,substep,timestep
0,0.0,0.0,0.0,0,0,0,1,0,0
1,5.0,0.0,0.0,0,0,0,1,1,1
2,10.0,1.0,1.0,0,0,0,1,1,2
3,15.0,2.0,1.0,0,0,0,1,1,3
4,20.0,3.0,2.0,0,0,0,1,1,4


# Language model connection

![Alt text](image.png)

In [3]:
import openai
import json
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Toolkit

In [5]:
from langchain.agents import create_pandas_dataframe_agent
# from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

In [6]:
# tools in the tool kit


def change_param(param,value):
    '''Changes the value of a parameter in the model'''
    # simulation.model.initial_state.update({
    # })
    value = float(value)
    simulation.model.params.update({
        param: [value]
    })
    experiment = Experiment(simulation)
    experiment.engine = Engine()
    result = experiment.run()
    # Convert the results to a pandas DataFrame
    globals()['df'] = pd.DataFrame(result)
    return f'new {param} value is {value} and the simulation dataframe is updated'

def model_info(param):
    '''Returns the information about the model'''
    if param in simulation.model.params:
        return f'{param} = {simulation.model.params[param]}'
    else:
        return f'{param} is not a parameter of the model'

# pandas agent as a tool

def analyze_dataframe(question):
    '''Analyzes the dataframe and returns the answer to the question'''
    pandas_agent = agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)
    answer = pandas_agent.run(question)
    
    return answer

In [7]:
# analyze_dataframe('what is the final value of the column distance')



> Entering new AgentExecutor chain...
Thought: I need to find the last row in the dataframe
Action: python_repl_ast
Action Input: df.tail(1)
Observation:     distance  coins  difficulty_factor  player_crashes  simulation  subset  \
40      85.0   23.0                8.0               1           0       0   

    run  substep  timestep  
40    1        1        40  
Thought: I now know the final answer
Final Answer: 85.0

> Finished chain.


'85.0'

In [8]:
# tool descriptions

function_descriptions_multiple = [
    {
        "name": "change_param",
        "description": "Changes the parameter of the cadcad simulation and returns dataframe as a global object. The parameter must be in this list:" + str(model.params.keys()),
        "parameters": {
            "type": "object",
            "properties": {
                "param": {
                    "type": "string",
                    "description": "parameter to change. choose from the list" + str(model.params.keys()),
                },
                "value": {
                    "type": "string",
                    "description": "value to change the parameter to, eg. 0.1",
                },
            },
            "required": ["param", "value"],
        },
    },
    {
        "name": "model_info",
        "description": "print current state of the simulation parameters",
        "parameters": {
            "type": "object",
            "properties": {
                "param": {
                    "type": "string",
                    "description": "type of information to print. choose from the list: " + str(model.params.keys()),
                },
            },
            "required": ["param"],
        },
    },
    {
        "name": "analyze_dataframe",
        "description": "Use this whenever a quantitative question is asked about the dataframe",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question asked by user that can be answered by an LLM dataframe agent",
                },
            },
            "required": ["question"],
        },
    },
]

# Agents

In [9]:
def executor_agent(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        # add function calling
        functions=function_descriptions_multiple,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output


# user_prompt = "whats the current value of crash chance?"
# print(executor_agent(user_prompt))

In [10]:
# user_prompt = "what is the avg value of the second row in the simulation?"
# print(executor_agent(user_prompt))

In [11]:
# user_prompt = "Change the parameter of crash chance to 20"
# answer = executor_agent(user_prompt)
# answer

In [12]:
def planner_agent(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {
            "role": "system",
            "content": '''
            You will be provided with a question by the user that is trying to run a cadcad python model. Your job is to provide the set of actions to take to get to the answer using the functions available.
            For example, if the user asks "if my crash chance parameter was 0.2, what would the avg coins be at the end of the simulation?" you reply with "### 1) we use the function change_param to change the crash chance parameter to 0.2,\n 2) use the function analyze_dataframe to get the avg coins at the end of the simulation. ###" 
            if the user asks "what would happen to the coins at the end of the simulation if my crash chance param was 10 perc lower?" you reply with "### 1) find out the current value of crash chance param using the model_info function,\n 2) we use function change_param to change the crash chance parameter to 0.1*crash_chance .\n 3) we use function analyze_dataframe to get the avg coins at the end of the simulation. ###"
            These are the functions available to you: {function_descriptions_multiple}. always remember to start and end plan with ###.
            '''
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
    )

    output = completion.choices[0].message
    return output

In [13]:
answer = planner_agent("whats the current value of crash chance, increase it to 20 and tell me the avg coins at the end of the simulation?")
print(answer.content)

### 1) To check the current value of the crash chance parameter, we can use the model_info function.
2) Use the change_param function to set the crash chance parameter to 20.
3) Finally, we can use the analyze_dataframe function to calculate the average coins at the end of the simulation. ###


# Orchestrator

In [20]:
from utils import plan_parser, print_color

In [21]:
def orchestrator_pipeline(user_input):
    plan = planner_agent(user_input).content
    plan_list = plan_parser(plan)
    print_color("Planner Agent:", "32")
    print('I have made a plan to follow: \n')

    for plan in plan_list:
        print(plan)

    print('\n')
    for plan in plan_list:
        print_color("Executor Agent:", "31")
        print('Thought: My task is to', plan)
        answer = executor_agent(plan)
        print('Action: I should call', answer.function_call.name,'function with these' , json.loads(answer.function_call.arguments),'arguments')
        if answer.function_call.name == 'analyze_dataframe':
            print_color("Analyzer Agent:", "34")
        print('Observation: I got this answer', eval(answer.function_call.name)(**json.loads(answer.function_call.arguments)))


In [19]:
orchestrator_pipeline("whats the current value of crash chance, change it to 20, and tell me the final coins at the end of the simulation?")

Planner Agent:
I have made a plan to follow: 

 1) To find out the current value of the crash chance parameter, we can use the `model_info` function.
2) We can then use the `change_param` function to change the crash chance parameter to 20.
3) Finally, we can use the `analyze_dataframe` function to get the final coins at the end of the simulation. 


Executor Agent:
Thought: My task is to  1) To find out the current value of the crash chance parameter, we can use the `model_info` function.
Action: I should call model_info function with these {'param': 'crash_chance'} arguments
Observation: I got this answer crash_chance = [0.2]
Executor Agent:
Thought: My task is to 2) We can then use the `change_param` function to change the crash chance parameter to 20.
Action: I should call change_param function with these {'param': 'crash_chance', 'value': '20'} arguments
Observation: I got this answer new crash_chance value is 20.0 and the simulation dataframe is updated
Executor Agent:
Thought: M

In [ ]:
# execute code here 
# what i can do is make a list of known values and add them each time a task is done.